In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
import torch.optim as optim
import pywt
from mne.time_frequency import tfr_multitaper
from sklearn.model_selection import train_test_split
import torch.cuda as cuda
from torch.autograd import Variable
import math

from utils import create_dataloader,train,bandpower
from dataset import EEG
import wandb
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
# Now do your import
from config import *

In [2]:
# home directory + datasets folder
#C:\Users\Pongk\Desktop\Work\mi-project\EEG-python\dataset\recorded_EEG
path = RECORED_PATH
#path = "/root/EEG_Model/dataset/finetune_EEG/"
#subject to run
runs = [1]
#runs = [1,2,3,4,12,13,14,15,16,17]
#runs = [1]
#runs = [7,8,9,10]
subjects = [21]
#recorded eeg class
eeg = EEG(path, subjects, runs)
raw=eeg.data_to_raw()

print("Raw done")
# apply filter 
#iir_param = dict(order=5, ftype='butter', output='sos')
#raw=raw.notch_filter([50])
#raw=raw.filter(8,14,method = 'iir',iir_params=iir_param,phase='zero')
#raw.filter(8,13, method='fir', verbose="error")
#print("Filter done")
#raw=eeg.raw_ica()
#eeg.create_epochs()
data, sf = raw.get_data(), raw.info['sfreq']

print(data.shape)
print(sf)


/Users/pongkornsettasompop/Desktop/work/Motor-Imagery/EEG-python/dataset/recorded_EEG
Raw done
(9, 47445)
250.0


In [3]:
#raw = eeg.set_reference(raw,['CZ'])

In [4]:
#pick channel
# print(raw.info['ch_names'])
# raw = eeg.pickChannel(raw,['C3','C4','STIM MARKERS'])
# print(raw.info['ch_names'])
# print(len(raw.info['ch_names']))

In [5]:
X,y = eeg.raw_preprocess(raw,event_id=[2.0],rest_stage=False)

[3231, 7984, 12739, 17464, 22218, 26972, 31681, 36435, 41189, 45943]


In [6]:
#Feature extraction
bp = bandpower(X, sf, [8, 13])
mul_bp = bandpower(X,sf,[8,13],method='multitaper')
print('Average band power'+ str(bp))
print('Multitaper band'+ str(mul_bp))

Effective window size : 1.024 (s)


/Users/pongkornsettasompop/anaconda3/envs/mi/lib/python3.10/site-packages/mne/time_frequency/multitaper.py:151: RuntimeWarning: invalid value encountered in divide
  d_k = psd_iter / (


Average band power3924.3429399626975
Multitaper bandnan


In [7]:
print(X.shape)
print(X[0,0,:])
print(y)

X = eeg.apply_baseline(X)
X = np.transpose(X,(0,2,1))
print(X[0,0,:])
print(np.transpose(X,(0,2,1)).shape)

(10, 8, 1750)
[-0.40787173  0.23934568  0.40821856 ...  1.68567048  1.00960541
  0.53274711]
[1 1 1 1 1 1 1 1 1 1]
[-0.3540237  39.38733037 -8.32761988 -9.95921699  0.          0.
  0.          0.        ]
(10, 8, 1750)


epochs=eeg.epochs(raw,tmin=-2,tmax=5,baseline=(-2,0))
print(len(epochs.times))
#X = X[:, :,np.newaxis,:]
X, y = eeg.get_X_y(epochs)
#(250*5)
#normal version 
#X = X[:,:,(250*3):]

#new version
#X = X[:,:,(250*8):(250*13)]

print(X.shape)
print(type(X))
#new_X = np.concatenate((fixation_X,imagine_X),axis=2)
print(y)

In [ ]:
#Old version use X. New version use new_X

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y)
print('Train size',X_train.shape, y_train.shape)
print('Test size',X_test.shape, y_test.shape)
print(y_train)
print(y_test)


In [ ]:
batch_size = 1750

train_loader = create_dataloader(X_train, y_train, batch_size=batch_size)
test_loader = create_dataloader(X_test, y_test, batch_size=batch_size)

num_step =math.ceil(len(train_loader.dataset) / batch_size)

In [ ]:
#wand setup
#"weightname":"S12_3-9_fir8-14"
#name=f"CNN_S12_3-9_fir8-14",
wandb.login()
wand = wandb.init(
      # Set the project where this run will be logged
      project="Motor-Imagery-New", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"TransR2_S17_iir_-2-5_8-13_3ch", 
      # Track hyperparameters and run metadata
      # 0.0000001
      config={
      "learning_rate": 0.0000001,
      "architecture": "ConvNet",
      "dataset": "Recorded",
      "epochs": 100000,
      "weightname":"S17_TransformeR2_iir_-2-5_8-13_3ch",
      "num_step_per_epoch" : num_step, 
      }
    )

In [ ]:
from model import ConvNet,CNN2D,gamenet,ConvNet2
from torchsummary import summary
from transformer import Transformer
# net = ConvNet2()

device = 'cuda'
sequence_len=1750 # sequence length of time series
max_len=5000 # max time series sequence length 
n_head = 4 # number of attention head
n_layer = 2 # number of encoder layer
drop_prob = 0.1
d_model = 200 # number of dimension ( for positional embedding)
ffn_hidden = 512 # size of hidden layer before classification 
in_features = 3 # for univariate time series (1d), it must be adjusted for 1. 
n_classes = 2
net =  Transformer( in_features=in_features,
                     d_model=d_model,
                     details=False,
                     n_head=n_head,
                     max_len=max_len,
                     seq_len=sequence_len,
                     ffn_hidden=ffn_hidden,
                     n_layers=n_layer,
                     drop_prob=drop_prob,
                     n_classes=n_classes,
                     device=device
                     )
#summary(net, (2, 641),32)

# Training section

In [ ]:
from utils import train
config = wand.config
optimizer = optim.Adam(net.parameters(), lr=config.learning_rate)
criterion = nn.CrossEntropyLoss()
device = 'cuda'

train_loss,valid_loss,train_accuracy,valid_accuracy =train(
    model = net,
    loader_train = train_loader,
    loader_test = test_loader,
    vail_loader = None,
    optimizer = optimizer  ,
    criterion = criterion ,
    device = 'cuda',
    wand = wand
)


wandb.alert(
            title='Finish',
            text=f'Finishing training',
        )

In [ ]:

import torch
import numpy as np

from torchsummary import summary
from transformer import Transformer
 

sequence_len=1750 # sequence length of time series
max_len=5000 # max time series sequence length 
n_head = 4 # number of attention head
n_layer = 2 # number of encoder layer
drop_prob = 0.1
d_model = 200 # number of dimension ( for positional embedding)
ffn_hidden = 512 # size of hidden layer before classification 
in_features = 3 # for univariate time series (1d), it must be adjusted for 1. 
n_classes = 2
model =  Transformer( in_features=in_features,
                     d_model=d_model,
                     details=False,
                     n_head=n_head,
                     max_len=max_len,
                     seq_len=sequence_len,
                     ffn_hidden=ffn_hidden,
                     n_layers=n_layer,
                     drop_prob=drop_prob,
                     n_classes=n_classes,
                     device=device)

batch_size = 555

#summary(net, (2, 641),32)
#summary(model, input_size=(batch_size,sequence_len,feature))

input_ = torch.from_numpy(np.empty((batch_size,sequence_len,in_features))).float()

model(input_)

print(model)